## 코로나 바이러스(COVID19)의 영향으로 중국인 관광객이 얼마나 줄었을까?

#### 외국인 출입국 통계 데이터 구하기
* 한국 관광 데이터랩 : 빅데이터를 활용한 여행행태 분석, 관광 통계 및 실태 조사, 지역별 시장분석 서비스
* 방한 외래 관광객, 국민 해외 관광객의 통계자료
    * 성별, 목적별, 연령별, 교통수단별, 국적별 집계
    * 활용데이터: 월별 외래객 입국-목적별/국적별 통계 데이터

In [45]:
# kto_201901.xlsx 엑셀 데이터 불러오기
import pandas as pd

kto_201901 = pd.read_excel('./data/KTO/kto_201901.xlsx',
                   header=1,
                   usecols='A:G',
                   skipfooter=4)
kto_201901.head()

,국적,관광,상용,공용,유학/연수,기타,계
0,아시아주,765082,10837,1423,14087,125521,916950
1,일본,198805,2233,127,785,4576,206526
2,대만,86393,74,22,180,1285,87954
3,홍콩,34653,59,2,90,1092,35896
4,마카오,2506,2,0,17,45,2570


#### 데이터 전처리(Data Preprocessing) = 데이터 가공(Data Manipulation), 데이터 클렌징(Data Cleansing), 데이터 핸들링(Data Handling)
* 데이터 변수별로 값에 이상이 없는지 확인
* 결측값 처리, 이상치 처리, 변수 정규화, 파생변수 생성 등

* 01. 데이터 탐색(컬럼별 특징)

In [46]:
kto_201901.tail()

,국적,관광,상용,공용,유학/연수,기타,계
62,아프리카 기타,768,718,90,206,908,2690
63,기타대륙,33,4,0,1,16,54
64,국적미상,33,4,0,1,16,54
65,교포소계,0,0,0,0,15526,15526
66,교포,0,0,0,0,15526,15526


In [47]:
kto_201901.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   국적      67 non-null     object
 1   관광      67 non-null     int64 
 2   상용      67 non-null     int64 
 3   공용      67 non-null     int64 
 4   유학/연수   67 non-null     int64 
 5   기타      67 non-null     int64 
 6   계       67 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 3.8+ KB


In [48]:
kto_201901.describe()

,관광,상용,공용,유학/연수,기타,계
count,67.00000,67.000000,67.000000,67.000000,67.000000,67.000000
mean,26396.80597,408.208955,132.507463,477.462687,5564.208955,32979.194030
std,102954.04969,1416.040302,474.406339,2009.484800,17209.438418,122821.369969
min,0.00000,0.000000,0.000000,0.000000,16.000000,54.000000
25%,505.00000,14.500000,2.500000,17.500000,260.000000,927.000000
50%,1304.00000,45.000000,14.000000,43.000000,912.000000,2695.000000
75%,8365.00000,176.500000,38.000000,182.000000,2824.500000,14905.500000
max,765082.00000,10837.000000,2657.000000,14087.000000,125521.000000,916950.000000


In [49]:
# 각 칼럼에서 0인 부분을 필터링
condition = (kto_201901['관광'] == 0) \
                | (kto_201901['상용'] == 0) \
                | (kto_201901['공용'] == 0) \
                | (kto_201901['유학/연수'] == 0)
kto_201901[condition]

# 결론: 교포는 통계 집계시 기타 목적으로 분류

,국적,관광,상용,공용,유학/연수,기타,계
4,마카오,2506,2,0,17,45,2570
20,이스라엘,727,12,0,9,57,805
22,우즈베키스탄,1958,561,0,407,2828,5754
38,스위스,613,18,0,19,97,747
45,그리스,481,17,4,0,273,775
46,포르투갈,416,14,0,13,121,564
51,크로아티아,226,12,0,3,250,491
54,폴란드,713,10,0,27,574,1324
59,대양주 기타,555,3,4,0,52,614
63,기타대륙,33,4,0,1,16,54


* 데이터프레임에 기준년월 추가

In [50]:
kto_201901['기준년월'] = '2019-01'
kto_201901.head()
kto_201901.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   국적      67 non-null     object
 1   관광      67 non-null     int64 
 2   상용      67 non-null     int64 
 3   공용      67 non-null     int64 
 4   유학/연수   67 non-null     int64 
 5   기타      67 non-null     int64 
 6   계       67 non-null     int64 
 7   기준년월    67 non-null     object
dtypes: int64(6), object(2)
memory usage: 4.3+ KB


* 국적 데이터만 남기기

In [51]:
# 국적들 확인
kto_201901['국적'].unique()

array(['아시아주', '일본', '대만', '홍콩', '마카오', '태국', '말레이시아', '필리핀', '인도네시아',
       '싱가포르', '미얀마', '베트남', '인도', '스리랑카', '파키스탄', '방글라데시', '캄보디아', '몽골',
       '중국', '이란', '이스라엘', '터키', '우즈베키스탄', '카자흐스탄', 'GCC', '아시아 기타', '미주',
       '미국', '캐나다', '멕시코', '브라질', '미주 기타', '구주', '영국', '독일', '프랑스',
       '네덜란드', '스웨덴', '스위스', '이탈리아', '덴마크', '노르웨이', '벨기에', '오스트리아', '스페인',
       '그리스', '포르투갈', '핀란드', '아일랜드', '우크라이나', '러시아', '크로아티아', '루마니아',
       '불가리아', '폴란드', '구주 기타', '대양주', '오스트레일리아', '뉴질랜드', '대양주 기타',
       '아프리카주', '남아프리카공화국', '아프리카 기타', '기타대륙', '국적미상', '교포소계', '교포'],
      dtype=object)

In [52]:
# 대륙 목록 만들기
continents_list = ['아시아주', '미주', '구주','대양주', '아프리카주', '기타대륙', '교포소계']
continents_list

['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계']

In [53]:
# 대륙 목록에 해당하는 값 제외
condition = (kto_201901.국적.isin(continents_list) == False)
kto_201901_country = kto_201901[condition]
kto_201901_country['국적'].unique()

array(['일본', '대만', '홍콩', '마카오', '태국', '말레이시아', '필리핀', '인도네시아', '싱가포르',
       '미얀마', '베트남', '인도', '스리랑카', '파키스탄', '방글라데시', '캄보디아', '몽골', '중국',
       '이란', '이스라엘', '터키', '우즈베키스탄', '카자흐스탄', 'GCC', '아시아 기타', '미국',
       '캐나다', '멕시코', '브라질', '미주 기타', '영국', '독일', '프랑스', '네덜란드', '스웨덴',
       '스위스', '이탈리아', '덴마크', '노르웨이', '벨기에', '오스트리아', '스페인', '그리스', '포르투갈',
       '핀란드', '아일랜드', '우크라이나', '러시아', '크로아티아', '루마니아', '불가리아', '폴란드',
       '구주 기타', '오스트레일리아', '뉴질랜드', '대양주 기타', '남아프리카공화국', '아프리카 기타',
       '국적미상', '교포'], dtype=object)

In [54]:
kto_201901_country.shape

(60, 8)

In [55]:
# index 재설정
kto_201901_country_newindex = kto_201901_country.reset_index(drop=True)
kto_201901_country_newindex.head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월
0,일본,198805,2233,127,785,4576,206526,2019-01
1,대만,86393,74,22,180,1285,87954,2019-01
2,홍콩,34653,59,2,90,1092,35896,2019-01
3,마카오,2506,2,0,17,45,2570,2019-01
4,태국,34004,37,199,96,6998,41334,2019-01


In [56]:
kto_201901_country_newindex.shape

(60, 8)

* 대륙 컬럼 만들기

In [57]:
continents = ['아시아']*25 + ['아메리카']*5 + ['유럽']*23 + ['오세아니아']*3 + ['아프리카']*2 + ['기타대륙'] + ['교포']
print(continents)
print(len(continents))

['아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아시아', '아메리카', '아메리카', '아메리카', '아메리카', '아메리카', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '유럽', '오세아니아', '오세아니아', '오세아니아', '아프리카', '아프리카', '기타대륙', '교포']
60


In [58]:
kto_201901_country_newindex['대륙'] = continents
kto_201901_country_newindex.head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙
0,일본,198805,2233,127,785,4576,206526,2019-01,아시아
1,대만,86393,74,22,180,1285,87954,2019-01,아시아
2,홍콩,34653,59,2,90,1092,35896,2019-01,아시아
3,마카오,2506,2,0,17,45,2570,2019-01,아시아
4,태국,34004,37,199,96,6998,41334,2019-01,아시아


In [59]:
kto_201901_country_newindex.tail()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙
55,대양주 기타,555,3,4,0,52,614,2019-01,오세아니아
56,남아프리카공화국,368,9,1,6,616,1000,2019-01,아프리카
57,아프리카 기타,768,718,90,206,908,2690,2019-01,아프리카
58,국적미상,33,4,0,1,16,54,2019-01,기타대륙
59,교포,0,0,0,0,15526,15526,2019-01,교포


* 국적별 관광객 비율 살펴보기

In [60]:
# 한국 방문 외국인 중에서 관광목적으로 입국하는 비율을 국가별로 비교
# 관광객 비율(%) 컬럼을 만듦
kto_201901_country_newindex['관광객비율(%)'] = \
    round(kto_201901_country_newindex['관광'] / kto_201901_country_newindex['계'] * 100, 1)
kto_201901_country_newindex.head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%)
0,일본,198805,2233,127,785,4576,206526,2019-01,아시아,96.3
1,대만,86393,74,22,180,1285,87954,2019-01,아시아,98.2
2,홍콩,34653,59,2,90,1092,35896,2019-01,아시아,96.5
3,마카오,2506,2,0,17,45,2570,2019-01,아시아,97.5
4,태국,34004,37,199,96,6998,41334,2019-01,아시아,82.3


In [61]:
# 관광객비율(%) 컬럼으로 내림차순 정렬
kto_201901_country_newindex.sort_values(by='관광객비율(%)', ascending=False).head()

# 결론: 대만, 마카오, 홍콩, 일본, 대양주 기타 국적의 국내 방문 외국인 중 90% 이상이 우리나라를 관광 목적으로 방문

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%)
1,대만,86393,74,22,180,1285,87954,2019-01,아시아,98.2
3,마카오,2506,2,0,17,45,2570,2019-01,아시아,97.5
2,홍콩,34653,59,2,90,1092,35896,2019-01,아시아,96.5
0,일본,198805,2233,127,785,4576,206526,2019-01,아시아,96.3
55,대양주 기타,555,3,4,0,52,614,2019-01,오세아니아,90.4


In [62]:
# 관광객비율(%) 컬럼으로 오름차순 정렬
kto_201901_country_newindex.sort_values(by='관광객비율(%)', ascending=True).head()

# 방글라데시, 파키스탄, 캄보디아 국적의 외국인들은 관광 목적으로 방문하는 비율이 25%도 안 됨
# 즉 소득 수준이 낮은 나라에서는 관광 목적 비율이 대체로 낮음

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%)
59,교포,0,0,0,0,15526,15526,2019-01,교포,0.0
14,방글라데시,149,126,27,97,848,1247,2019-01,아시아,11.9
12,스리랑카,157,54,5,28,1043,1287,2019-01,아시아,12.2
13,파키스탄,238,178,10,193,413,1032,2019-01,아시아,23.1
15,캄보디아,635,39,55,51,1915,2695,2019-01,아시아,23.6


In [63]:
# 대륙별 관광객 비율의 평균
kto_201901_country_newindex.pivot_table(index='대륙',
                                        values='관광객비율(%)',
                                        aggfunc='mean')

# 평균 관광객 비율이 가장 높은 대륙은 오세아니아
# 반대로 아프리카에서 우리나라로 방문하는 외국인은 관광목적으로 방문하는 사람이 32.7%로 적음
# 의외로 거리가 가까운 국가의 관광객 비율이 아메리카나 유럽보다 낮음

,관광객비율(%)
대륙,
교포,0.000000
기타대륙,61.100000
아메리카,68.200000
아시아,59.624000
아프리카,32.700000
오세아니아,84.833333
유럽,63.826087


In [64]:
# 중국 국적만 필터링
condition = (kto_201901_country_newindex['국적'] == '중국')
kto_201901_country_newindex[condition]

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%)
17,중국,320113,2993,138,8793,60777,392814,2019-01,아시아,81.5


* 기준년월별로 전체 외국인 관광객 대비 국적별 관광객 비율 살펴보기

In [65]:
# 2019년 1월 우리나라 방문하는 전체 외국인 관광객 숫자 구하기
tourist_sum = sum(kto_201901_country_newindex['관광'])
tourist_sum

884293

In [66]:
kto_201901_country_newindex['전체비율(%)'] = \
    round(kto_201901_country_newindex['관광'] / tourist_sum * 100, 1)
kto_201901_country_newindex.head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
0,일본,198805,2233,127,785,4576,206526,2019-01,아시아,96.3,22.5
1,대만,86393,74,22,180,1285,87954,2019-01,아시아,98.2,9.8
2,홍콩,34653,59,2,90,1092,35896,2019-01,아시아,96.5,3.9
3,마카오,2506,2,0,17,45,2570,2019-01,아시아,97.5,0.3
4,태국,34004,37,199,96,6998,41334,2019-01,아시아,82.3,3.8


In [67]:
kto_201901_country_newindex.sort_values('전체비율(%)', ascending=False).head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
17,중국,320113,2993,138,8793,60777,392814,2019-01,아시아,81.5,36.2
0,일본,198805,2233,127,785,4576,206526,2019-01,아시아,96.3,22.5
1,대만,86393,74,22,180,1285,87954,2019-01,아시아,98.2,9.8
25,미국,42989,418,2578,229,16523,62737,2019-01,아메리카,68.5,4.9
2,홍콩,34653,59,2,90,1092,35896,2019-01,아시아,96.5,3.9


## 데이터 전처리 과정 함수로 만들기

In [68]:
def create_kto_data(yy, mm):
    file_path = './data/KTO/kto_{}{}.xlsx'.format(yy, mm)
    df = pd.read_excel(file_path, header=1, skipfooter=4, usecols='A:G')
    
    # "기준년월" 컬럼 추가
    df['기준년월'] = '{}-{}'.format(yy, mm)
    
    # "국적" 컬럼에서 대륙 제거하고 국가만 남기기
    ignort_list = ['아시아주', '미주', '구주', '대양주', '아프리카주', '기타대륙', '교포소계'] # 제거할 대륙명 선정
    condition = (df['국적'].isin(ignort_list) == False)
    df_country = df[condition].reset_index(drop=True)
    
    # "대륙" 컬럼 추가
    continents = ['아시아']*25 + ['아메리카']*5 + ['유럽']*23 + ['오세아니아']*3 + ['아프리카']*2 + ['기타대륙'] + ['교포']
    df_country['대륙'] = continents
    
    # 국가별 "관광객비율(%)" 컬럼 추가
    df_country['관광객비율(%)'] = round(df_country.관광 / df_country.계 * 100, 1)
    
    # "전체비율(%)" 컬럼 추가
    tourist_sum = sum(df_country['관광'])
    df_country['전체비율(%)'] = round(df_country['관광'] / tourist_sum * 100, 1)
    
    return(df_country)

In [69]:
# create_kto_data() 함수를 활용해서 2018년 12월 데이터 불러오기
kto_test = create_kto_data(2018, 12)
kto_test.head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
0,일본,252461,1698,161,608,3593,258521,2018-12,아시아,97.7,22.7
1,대만,85697,71,22,266,1252,87308,2018-12,아시아,98.2,7.7
2,홍콩,58355,41,3,208,939,59546,2018-12,아시아,98.0,5.2
3,마카오,6766,0,1,20,36,6823,2018-12,아시아,99.2,0.6
4,태국,47242,42,302,58,6382,54026,2018-12,아시아,87.4,4.2


## 다수의 엑셀 데이터 불러와서 합치기

In [70]:
for yy in range(2010, 2021):
    for mm in range(1, 13):
        yymm = '{}{}'.format(yy, mm)
        print(yymm)

20101
20102
20103
20104
20105
20106
20107
20108
20109
201010
201011
201012
20111
20112
20113
20114
20115
20116
20117
20118
20119
201110
201111
201112
20121
20122
20123
20124
20125
20126
20127
20128
20129
201210
201211
201212
20131
20132
20133
20134
20135
20136
20137
20138
20139
201310
201311
201312
20141
20142
20143
20144
20145
20146
20147
20148
20149
201410
201411
201412
20151
20152
20153
20154
20155
20156
20157
20158
20159
201510
201511
201512
20161
20162
20163
20164
20165
20166
20167
20168
20169
201610
201611
201612
20171
20172
20173
20174
20175
20176
20177
20178
20179
201710
201711
201712
20181
20182
20183
20184
20185
20186
20187
20188
20189
201810
201811
201812
20191
20192
20193
20194
20195
20196
20197
20198
20199
201910
201911
201912
20201
20202
20203
20204
20205
20206
20207
20208
20209
202010
202011
202012


In [71]:
# test
mm = 1
print(mm)
print(str(mm).zfill(2))
print(str(mm).zfill(3))
print(str(mm).zfill(4))

1
01
001
0001


In [72]:
# 6자리로 정렬하여 기준년월 출력하기
for yy in range(2010, 2021):
    for mm in range(1, 13):
        mm_str = str(mm).zfill(2)
        yymm = '{}{}'.format(yy, mm_str)
        print(yymm)

201001
201002
201003
201004
201005
201006
201007
201008
201009
201010
201011
201012
201101
201102
201103
201104
201105
201106
201107
201108
201109
201110
201111
201112
201201
201202
201203
201204
201205
201206
201207
201208
201209
201210
201211
201212
201301
201302
201303
201304
201305
201306
201307
201308
201309
201310
201311
201312
201401
201402
201403
201404
201405
201406
201407
201408
201409
201410
201411
201412
201501
201502
201503
201504
201505
201506
201507
201508
201509
201510
201511
201512
201601
201602
201603
201604
201605
201606
201607
201608
201609
201610
201611
201612
201701
201702
201703
201704
201705
201706
201707
201708
201709
201710
201711
201712
201801
201802
201803
201804
201805
201806
201807
201808
201809
201810
201811
201812
201901
201902
201903
201904
201905
201906
201907
201908
201909
201910
201911
201912
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012


In [73]:
# 데이터 합하기
df = pd.DataFrame()
df_list = []

for yy in range(2010, 2021):
    for mm in range(1, 13):
        try:
            temp = create_kto_data(str(yy), str(mm).zfill(2))
            df_list.append(temp)
        except:
            pass
        
df = pd.concat(df_list, ignore_index=True)
df.head()
        

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
0,일본,202825,1750,89,549,3971,209184,2010-01,아시아,97.0,50.6
1,대만,35788,41,17,37,516,36399,2010-01,아시아,98.3,8.9
2,홍콩,13874,55,0,21,595,14545,2010-01,아시아,95.4,3.5
3,마카오,554,0,0,0,0,554,2010-01,아시아,100.0,0.1
4,태국,13374,39,13,53,4335,17814,2010-01,아시아,75.1,3.3


In [74]:
df.tail()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
7495,대양주 기타,1,0,1,0,3,5,2020-05,오세아니아,20.0,0.0
7496,남아프리카공화국,1,0,3,0,25,29,2020-05,아프리카,3.4,0.0
7497,아프리카 기타,3,1,3,6,118,131,2020-05,아프리카,2.3,0.0
7498,국적미상,1,0,0,0,3,4,2020-05,기타대륙,25.0,0.0
7499,교포,0,0,0,0,790,790,2020-05,교포,0.0,0.0


## 통합 데이터를 엑셀 파일로 저장하기

In [75]:
df.to_excel('./Data/KTO/kto_total.xlsx', index=False)

## 국적별 필터링된 데이터를 엑셀 파일로 저장

In [76]:
# 중국인 관광객만 필터링하기
condition = (df['국적'] == '중국')
df_filter = df[condition]
df_filter.head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
17,중국,40425,11930,55,2751,36091,91252,2010-01,아시아,44.3,10.1
77,중국,60590,7907,68,29546,42460,140571,2010-02,아시아,43.1,13.6
137,중국,50330,13549,174,14924,62480,141457,2010-03,아시아,35.6,9.2
197,중국,84252,13306,212,2199,47711,147680,2010-04,아시아,57.1,15.5
257,중국,89056,12325,360,2931,49394,154066,2010-05,아시아,57.8,17.0


In [77]:
file_path = './Data/KTO/[국적별 관광객 데이터] 중국.xlsx'
df_filter.to_excel(file_path, index=False)

In [78]:
cntry_list = df['국적'].unique()
cntry_list

array(['일본', '대만', '홍콩', '마카오', '태국', '말레이시아', '필리핀', '인도네시아', '싱가포르',
       '미얀마', '베트남', '인도', '스리랑카', '파키스탄', '방글라데시', '캄보디아', '몽골', '중국',
       '이란', '이스라엘', '터키', '우즈베키스탄', '카자흐스탄', 'GCC', '아시아 기타', '미국',
       '캐나다', '멕시코', '브라질', '미주 기타', '영국', '독일', '프랑스', '네덜란드', '스웨덴',
       '스위스', '이탈리아', '덴마크', '노르웨이', '벨기에', '오스트리아', '스페인', '그리스', '포르투갈',
       '핀란드', '아일랜드', '우크라이나', '러시아', '크로아티아', '루마니아', '불가리아', '폴란드',
       '구주 기타', '오스트레일리아', '뉴질랜드', '대양주 기타', '남아프리카공화국', '아프리카 기타',
       '국적미상', '교포'], dtype=object)

In [79]:
for cntry in cntry_list:
    condition = (df['국적'] == cntry)
    df_filter = df[condition]
    
    file_path = './Data/KTO/[국적별 관광객 데이터] {}.xlsx'.format(cntry)
    df_filter.to_excel(file_path, index=False)

In [80]:
import pandas as pd

total = pd.read_excel('./Data/KTO/kto_total.xlsx')
total.head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
0,일본,202825,1750,89,549,3971,209184,2010-01,아시아,97.0,50.6
1,대만,35788,41,17,37,516,36399,2010-01,아시아,98.3,8.9
2,홍콩,13874,55,0,21,595,14545,2010-01,아시아,95.4,3.5
3,마카오,554,0,0,0,0,554,2010-01,아시아,100.0,0.1
4,태국,13374,39,13,53,4335,17814,2010-01,아시아,75.1,3.3


In [81]:
total['기준년월'] = pd.to_datetime(total['기준년월'], format = '%Y-%m')
total['기준년월'] = total['기준년월'].dt.to_period('M')
total.head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
0,일본,202825,1750,89,549,3971,209184,2010-01,아시아,97.0,50.6
1,대만,35788,41,17,37,516,36399,2010-01,아시아,98.3,8.9
2,홍콩,13874,55,0,21,595,14545,2010-01,아시아,95.4,3.5
3,마카오,554,0,0,0,0,554,2010-01,아시아,100.0,0.1
4,태국,13374,39,13,53,4335,17814,2010-01,아시아,75.1,3.3


In [82]:
# 중국만 필터링
china = total[total['국적'] == '중국']
china.head()

,국적,관광,상용,공용,유학/연수,기타,계,기준년월,대륙,관광객비율(%),전체비율(%)
17,중국,40425,11930,55,2751,36091,91252,2010-01,아시아,44.3,10.1
77,중국,60590,7907,68,29546,42460,140571,2010-02,아시아,43.1,13.6
137,중국,50330,13549,174,14924,62480,141457,2010-03,아시아,35.6,9.2
197,중국,84252,13306,212,2199,47711,147680,2010-04,아시아,57.1,15.5
257,중국,89056,12325,360,2931,49394,154066,2010-05,아시아,57.8,17.0


In [83]:
# 2019년을 기준으로 이전과 이후 변수 각각 분류
china_before = china[(china['기준년월'] > '2019-05') & (china['기준년월'] < '2019-12')]
china_after = china[china['기준년월'] >= '2019-12']

In [84]:
# 관광객 수만 필터링
china_before = china_before['관광']
china_after = china_after['관광']

In [85]:
# 등분산성 검정
from scipy import stats
levene_stat, levene_p_value = stats.levene(china_before, china_after)
equal_var = levene_p_value > 0.05

print(f"등분산성 여부: {levene_p_value:.4f} -> {'등분산' if equal_var else '이분산'} 가정")

등분산성 여부: 0.1312 -> 등분산 가정


In [86]:
# t검정 수행
t_stat, p_value = stats.ttest_ind(china_before, china_after, equal_var=True)

print(f"T-통계량: {t_stat:.4f}")
print(f"P-값: {p_value:.4f}")

T-통계량: 3.3925
P-값: 0.0069


In [90]:
# t검정 함수 정의
def ttest_nationality(nationality, total, date_col='기준년월', tourism_col='관광'):
    # 국적 필터링
    nationality_filter = total[total['국적'] == nationality]

    # 코로나19 전후 데이터 필터링
    before_covid = nationality_filter[(nationality_filter[date_col] > '2019-05') &
                                      (nationality_filter[date_col] < '2019-12')]
    after_covid = nationality_filter[nationality_filter[date_col] >= '2019-12']
    
    # 관광객 수만 필터링
    before_covid_tourism = before_covid[tourism_col]
    after_covid_tourism = after_covid[tourism_col]
    
    # 등분산성 검정
    levene_stat, levene_p_value = stats.levene(before_covid_tourism, after_covid_tourism)
    equal_var = levene_p_value > 0.05
    
    # t검정 수행
    t_stat, p_value = stats.ttest_ind(before_covid_tourism, after_covid_tourism, equal_var=equal_var)
    
    print(f"국적: {nationality}")
    print(f"등분산성 여부: {levene_p_value} -> {'등분산' if equal_var else '이분산'} 가정")
    print(f"T-통계량: {t_stat:.4f}")
    print(f"P-값: {p_value:.4f}")
    print('\n')
    
# 국적 리스트
nationalities = ['중국', '대만', '홍콩', '일본', '미국']

# 반복문으로 각 국적에 대한 t검정 수행
for nationality in nationalities:
    ttest_nationality(nationality, total)

국적: 중국
등분산성 여부: 0.13117380080232627 -> 등분산 가정
T-통계량: 3.3925
P-값: 0.0069


국적: 대만
등분산성 여부: 0.00887159584094535 -> 이분산 가정
T-통계량: 3.3766
P-값: 0.0175


국적: 홍콩
등분산성 여부: 0.06651578397903857 -> 등분산 가정
T-통계량: 2.2874
P-값: 0.0452


국적: 일본
등분산성 여부: 2.1060648801132006e-05 -> 이분산 가정
T-통계량: 3.1493
P-값: 0.0218


국적: 미국
등분산성 여부: 0.060623697304497376 -> 등분산 가정
T-통계량: 4.7884
P-값: 0.0007


